In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.DataFrame(
    dict(
        user_id=np.random.randint(0, 1000, size=100000),
        n_imp=np.random.randint(1, 1001, size=100000),
    )
)
df["n_click"] = df.n_imp.apply(lambda x: np.random.randint(0, x+1))
df

,user_id,n_imp,n_click
0,950,413,51
1,827,527,465
2,231,341,329
3,568,446,159
4,856,569,541
...,...,...,...
99995,987,196,165
99996,372,373,200
99997,591,809,53
99998,173,109,94


In [3]:
from sklearn.model_selection import train_test_split

user_ids = df.user_id.unique()

def data_loader(i: int) -> list[pd.DataFrame]:
    return [
        df[df.user_id.isin(uids)]
        for uids in train_test_split(user_ids, test_size=.5, random_state=i)
    ]

In [4]:
from t_ab.ctr import ImpressionBasedCTRTtest, UserBasedCTRTtest

_ibtest = ImpressionBasedCTRTtest("user_id", "n_imp")
_ubtest = UserBasedCTRTtest("user_id", "n_imp")


def ibtest(dfs: list[pd.DataFrame]) -> float:
    return _ibtest(*dfs, "n_click")[1].pvalue


def ubtest(dfs: list[pd.DataFrame]) -> float:
    return _ubtest(*dfs, "n_click")[1].pvalue

In [5]:
from t_ab.aa import AATest

In [6]:
results = AATest()(data_loader, ibtest, ubtest)

In [7]:
for result in results:
    print(result.pvalue, result.is_rejected)

0.8578671947051901 False
0.9844590021345854 False
